<a href="https://colab.research.google.com/github/sidrampotdar/APP/blob/master/Copy_of_Mindora_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain_groq langchain_core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


# New Section

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Hello?")
print(result.content)

Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?


In [4]:
!pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.3 MB/s eta 0:00:00


In [5]:
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00


In [6]:
!pip install sentence_transformers


In [7]:
!pip install --upgrade langchain langchain-groq


Example of Bot with Conversation History Feature

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    # Ensure the 'data' directory exists and contains PDF files
    loader = DirectoryLoader("/content/sample_data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()

    print("ChromaDB created and data saved")
    return vector_db

class ConversationHistory:
    def __init__(self):
        self.history = []

    def add_message(self, message_type, message):
        self.history.append((message_type, message))

    def get_context(self):
        context = "\n".join([f"{t}: {m}" for t, m in self.history])
        return context if context else "No prior conversation history."

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    prompt_template = """You are a compassionate mental health chatbot. Respond thoughtfully to the following question, considering the conversation history:
    {context}
    User: {question}
    Chatbot:"""

    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

def main():
    print("Initializing Chatbot.........")
    llm = initialize_llm()

    db_path = "./chroma_db"
    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    conversation_history = ConversationHistory()  # Initialize history *outside* the loop
    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        query = input("\nHuman: ")
        if query.lower() == "exit":
            print("Chatbot: Take Care of yourself, Goodbye!")
            break

        conversation_history.add_message("Human", query)
        context = conversation_history.get_context()  # Get updated conversation history
        # Pass context and query correctly to the chain
        response = qa_chain({"query": f"{context}\nUser: {query}"})
        chatbot_reply = response['result']
        conversation_history.add_message("Chatbot", chatbot_reply)  # Save chatbot's response
        print(f"Chatbot: {chatbot_reply}")

if __name__ == "__main__":
    main()


Initializing Chatbot.........


FileNotFoundError: Directory not found: '/content/data/'

In [ ]:
!pip install gradio

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


print("Intializing Chatbot.........")
llm = initialize_llm()

db_path = "/content/chroma_db"

if not os.path.exists(db_path):
  vector_db  = create_vector_db()
else:
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history = []):
  if not user_input.strip():
    return "Please provide a valid input", history
  response = qa_chain.run(user_input)
  history.append((user_input, response))
  return "", history

with gr.Blocks(theme = 'Respair/Shiki@1.2.1') as app:
    gr.Markdown("# 🧠 Mental Health Chatbot 🤖")
    gr.Markdown("A compassionate chatbot designed to assist with mental well-being. Please note: For serious concerns, contact a professional.")

    chatbot = gr.ChatInterface(fn=chatbot_response, title="Mental Health Chatbot")

    gr.Markdown("This chatbot provides general support. For urgent issues, seek help from licensed professionals.")

app.launch()

In [ ]:
!pip install firebase-admin


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload the serviceAccountKey.json file


In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)


Integration of DB(FireBase) and Email Sending Feature without Conversation HIstory Feature

In [ ]:
import datetime
import firebase_admin
from firebase_admin import credentials, firestore
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Firebase Connection Setup
def setup_firebase():
    if not firebase_admin._apps:
        cred = credentials.Certificate("/content/project141-22a01-firebase-adminsdk-sgnh9-181e811b93.json")
        firebase_admin.initialize_app(cred)
    db = firestore.client()
    return db

# Emotion Detection Function
def detect_emotion(user_input):
    model_name = "j-hartmann/emotion-english-distilroberta-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
    emotion = emotion_labels[predicted_class]
    confidence = torch.softmax(logits, dim=1)[0][predicted_class].item()
    return emotion, confidence

# Helper function to send email
email_sent_users = set()

def send_email(user_email, city):
    global email_sent_users  # Reference the global set

    if user_email in email_sent_users:
        print(f"Email already sent to {user_email}.")
        return  # Exit the function
    helpline_numbers = {
        'mumbai': 'Mpower 1on1: 1800-120-820050',
        'pune': 'Connecting NGO: +91-9922001122',
        'nagpur': 'Jeevan Jyoti Suicide Prevention Centre: +91-712-2775555',
        'nashik': 'Connecting NGO: +91-9922001122',
        'aurangabad': 'Connecting NGO: +91-9922001122',
        'solapur': 'Connecting NGO: +91-9922001122',
        'kolhapur': 'Tele-MANAS: 14416',
        'thane': 'Tele-MANAS: 14416',
        'amravati': 'Tele-MANAS: 14416',
        'nanded': 'Tele-MANAS: 14416',
        'sangli': 'Tele-MANAS: 14416',
        'jalgaon': 'Tele-MANAS: 14416',
        'akola': 'Tele-MANAS: 14416',
        'latur': 'Tele-MANAS: 14416',
        'dhule': 'Tele-MANAS: 14416',
        'malegaon': 'Tele-MANAS: 14416',
        'bhusawal': 'Tele-MANAS: 14416',
        'beed': 'Tele-MANAS: 14416',
        'chandrapur': 'Tele-MANAS: 14416',
        'parbhani': 'Tele-MANAS: 14416',
        'yavatmal': 'Tele-MANAS: 14416',
        'osmanabad': 'Tele-MANAS: 14416',
        'navi mumbai': 'Mpower 1on1: 1800-120-820050',
        'kalyan': 'Mpower 1on1: 1800-120-820050',
        'ulhasnagar': 'Mpower 1on1: 1800-120-820050',
        'ambarnath': 'Mpower 1on1: 1800-120-820050',
        'dombivli': 'Mpower 1on1: 1800-120-820050',
        'bhiwandi': 'Mpower 1on1: 1800-120-820050',
        'ratnagiri': 'Mpower 1on1: 1800-120-820050',
        'satara': 'Mpower 1on1: 1800-120-820050',
        'sindhudurg': 'Mpower 1on1: 1800-120-820050',
        'wardha': 'Mpower 1on1: 1800-120-820050',
        'gondia': 'Mpower 1on1: 1800-120-820050',
        'bhandara': 'Mpower 1on1: 1800-120-820050',
        'washim': 'Mpower 1on1: 1800-120-820050',
        'hingoli': 'Mpower 1on1: 1800-120-820050',
        'gadchiroli': 'Mpower 1on1: 1800-120-820050',
    }

    if city not in helpline_numbers:
        helpline_numbers[city] = 'No helpline data available for your city.'

    sender_email = "pruthviraj.22420262@viit.ac.in"
    sender_password = "22420262"
    receiver_email = user_email

    subject = "Mental Health Support: Helpline Number"
    body = f"""
    Dear {user_email},

    We noticed that you're going through a tough time. Your emotional well-being is important to us.

    Here is the helpline number for your city ({city}):
    {helpline_numbers[city]}

    Please reach out to the helpline for any support you may need.

    Take care,
    Your Mental Health Chatbot
    """

    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        text = message.as_string()
        server.sendmail(sender_email, receiver_email, text)
        email_sent_users.add(receiver_email)
        print(f"Email sent successfully to {receiver_email}")
        server.quit()
    except Exception as e:
        print(f"Error sending email: {e}")

# Vector Database Creation
def create_vector_db():
    loader = DirectoryLoader("/content/data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

# QA Chain Setup
class ConversationHistory:
    def __init__(self):
        self.history = []

    def add_message(self, message_type, message):
        """Adds a message to the conversation history."""
        self.history.append((message_type, message))

    def get_context(self):
        """Retrieves the conversation history formatted as context."""
        return "\n".join([f"{t}: {m}" for t, m in self.history]) if self.history else "No prior conversation history."


def setup_qa_chain(vector_db, llm, user_info, conversation_history):
    retriever = vector_db.as_retriever()

    prompt_template = f"""
    You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    User Info:
    Name: {user_info['name']}
    Age: {user_info['age']}
    City: {user_info['city']}
    Gender: {user_info['gender']}

    Previous Conversation question:
    {query}


    Context: {{context}}
    User: {{question}}
    Chatbot: """

    PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# Store User Data in Firestore
def store_user_data(db, user_info):
    user_ref = db.collection("users").document(user_info['name'])
    user_ref.set(user_info)

# Store Chat Data in Firestore
def store_chat_data(db, user_info, query, response, emotion, confidence):
    user_chat_ref = db.collection("users").document(user_info['name']).collection("chat_history")

    chat_entry = {
        "user_name": user_info['name'],
        "user_query": query,
        "chatbot_response": response,
        "emotion_detected": emotion,
        "confidence": confidence,
        "timestamp": datetime.datetime.now()
    }

    user_chat_ref.add(chat_entry)

# Main Chatbot Function
def main():
    print("Initializing Chatbot.........")
    db = setup_firebase()

    # Initialize the LLM
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )

    db_path = "/content/chroma_db"

    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    user_info = {}
    print("Welcome! Before we start, please provide some information about yourself.")
    # user_info['name'] = input("What is your name? ")
    # user_info['age'] = input("What is your age? ")
    # user_info['city'] = input("What is your city? ")
    # user_info['gender'] = input("What is your gender? ")
    # user_info['email'] = input("Please provide your email address for support: ")

    ########

    user_info['name'] = "ex"
    user_info['age'] = "ex"
    user_info['city'] = "ex"
    user_info['gender'] = "ex"
    user_info['email'] = "example@gmail.com"

    store_user_data(db, user_info)

    # Initialize conversation history
    conversation_history = ""

    qa_chain = setup_qa_chain(vector_db, llm, user_info, conversation_history)

    while True:
        query = input("\nHuman: ")
        if query.lower() == "exit":
            print("Chatbot: Take care of yourself, Goodbye!")
            break

        emotion, confidence = detect_emotion(query)
        print(f"Emotion Detected: {emotion} (Confidence: {confidence:.2f})")

        if emotion == "sadness" and confidence > 0.75:
            send_email(user_info['email'], user_info['city'])

        # Update conversation history and get response
        response = qa_chain.run(query)
        conversation_history += f"\nHuman: {query}\nChatbot: {response}"

        print(f"Chatbot: {response}")
        store_chat_data(db, user_info, query, response, emotion, confidence)

if __name__ == "__main__":
    main()



Integration of DB(FireBase) and Email Sending Feature with Conversation HIstory Feature

In [ ]:
import datetime
import firebase_admin
from firebase_admin import credentials, firestore
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Firebase Connection Setup
def setup_firebase():
    if not firebase_admin._apps:
        cred = credentials.Certificate("/content/project141-22a01-firebase-adminsdk-sgnh9-181e811b93.json")
        firebase_admin.initialize_app(cred)
    db = firestore.client()
    return db

# Emotion Detection Function
def detect_emotion(user_input):
    model_name = "j-hartmann/emotion-english-distilroberta-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
    emotion = emotion_labels[predicted_class]
    confidence = torch.softmax(logits, dim=1)[0][predicted_class].item()
    return emotion, confidence

# Helper function to send email
email_sent_users = set()

def send_email(user_email, city):
    global email_sent_users  # Reference the global set

    if user_email in email_sent_users:
        print(f"Email already sent to {user_email}.")
        return  # Exit the function
    helpline_numbers = {
        'mumbai': 'Mpower 1on1: 1800-120-820050',
        'pune': 'Connecting NGO: +91-9922001122',
        'nagpur': 'Jeevan Jyoti Suicide Prevention Centre: +91-712-2775555',
        'nashik': 'Connecting NGO: +91-9922001122',
        'aurangabad': 'Connecting NGO: +91-9922001122',
        'solapur': 'Connecting NGO: +91-9922001122',
        'kolhapur': 'Tele-MANAS: 14416',
        'thane': 'Tele-MANAS: 14416',
        'amravati': 'Tele-MANAS: 14416',
        'nanded': 'Tele-MANAS: 14416',
        'sangli': 'Tele-MANAS: 14416',
        'jalgaon': 'Tele-MANAS: 14416',
        'akola': 'Tele-MANAS: 14416',
        'latur': 'Tele-MANAS: 14416',
        'dhule': 'Tele-MANAS: 14416',
        'malegaon': 'Tele-MANAS: 14416',
        'bhusawal': 'Tele-MANAS: 14416',
        'beed': 'Tele-MANAS: 14416',
        'chandrapur': 'Tele-MANAS: 14416',
        'parbhani': 'Tele-MANAS: 14416',
        'yavatmal': 'Tele-MANAS: 14416',
        'osmanabad': 'Tele-MANAS: 14416',
        'navi mumbai': 'Mpower 1on1: 1800-120-820050',
        'kalyan': 'Mpower 1on1: 1800-120-820050',
        'ulhasnagar': 'Mpower 1on1: 1800-120-820050',
        'ambarnath': 'Mpower 1on1: 1800-120-820050',
        'dombivli': 'Mpower 1on1: 1800-120-820050',
        'bhiwandi': 'Mpower 1on1: 1800-120-820050',
        'ratnagiri': 'Mpower 1on1: 1800-120-820050',
        'satara': 'Mpower 1on1: 1800-120-820050',
        'sindhudurg': 'Mpower 1on1: 1800-120-820050',
        'wardha': 'Mpower 1on1: 1800-120-820050',
        'gondia': 'Mpower 1on1: 1800-120-820050',
        'bhandara': 'Mpower 1on1: 1800-120-820050',
        'washim': 'Mpower 1on1: 1800-120-820050',
        'hingoli': 'Mpower 1on1: 1800-120-820050',
        'gadchiroli': 'Mpower 1on1: 1800-120-820050',
    }

    if city not in helpline_numbers:
        helpline_numbers[city] = 'No helpline data available for your city.'

    sender_email = "pruthviraj.22420262@viit.ac.in"
    sender_password = "22420262"
    receiver_email = user_email

    subject = "Mental Health Support: Helpline Number"
    body = f"""
    Dear {user_email},

    We noticed that you're going through a tough time. Your emotional well-being is important to us.

    Here is the helpline number for your city ({city}):
    {helpline_numbers[city]}

    Please reach out to the helpline for any support you may need.

    Take care,
    Your Mental Health Chatbot
    """

    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        text = message.as_string()
        server.sendmail(sender_email, receiver_email, text)
        email_sent_users.add(receiver_email)
        print(f"Email sent successfully to {receiver_email}")
        server.quit()
    except Exception as e:
        print(f"Error sending email: {e}")

# Vector Database Creation
def create_vector_db():
    loader = DirectoryLoader("/content/data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

# QA Chain Setup
class ConversationHistory:
    def __init__(self):
        self.history = []

    def add_message(self, message_type, message):
        self.history.append((message_type, message))

    def get_context(self):
        context = "\n".join([f"{t}: {m}" for t, m in self.history])
        return context if context else "No prior conversation history."

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    prompt_template = """You are a compassionate mental health chatbot. Respond thoughtfully to the following question, considering the conversation history:
    {context}
    User: {question}
    Chatbot:"""

    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

def store_user_data(db, user_info):
    user_ref = db.collection("users").document(user_info['name'])
    user_ref.set(user_info)

def store_chat_data(db, user_info, query, response, emotion, confidence):
    user_chat_ref = db.collection("users").document(user_info['name']).collection("chat_history")

    chat_entry = {
        "user_name": user_info['name'],
        "user_query": query,
        "chatbot_response": response,
        "emotion_detected": emotion,
        "confidence": confidence,
        "timestamp": datetime.datetime.now()
    }

    user_chat_ref.add(chat_entry)

def main():
    print("Initializing Chatbot.........")
    db = setup_firebase()

    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )

    db_path = "/content/chroma_db"

    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    user_info = {}
    print("Welcome! Before we start, please provide some information about yourself.")
    user_info['name'] = input("What is your name? ")
    user_info['age'] = input("What is your age? ")
    user_info['city'] = input("What is your city? ")
    user_info['gender'] = input("What is your gender? ")
    user_info['email'] = input("Please provide your email address for support: ")

# ... (rest of your code)
    store_user_data(db, user_info)

    conversation_history = ConversationHistory()  # Initialize history
    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        query = input("\nHuman: ")
        if query.lower() == "exit":
            print("Chatbot: Take care of yourself, Goodbye!")
            break

        emotion, confidence = detect_emotion(query)
        print(f"Emotion Detected: {emotion} (Confidence: {confidence:.2f})")

        if emotion == "sadness" and confidence > 0.75:
            send_email(user_info['email'], user_info['city'])

        conversation_history.add_message("Human", query)
        context = conversation_history.get_context()
        response = qa_chain({"query": f"{context}\nUser: {query}"})
        chatbot_reply = response['result']
        conversation_history.add_message("Chatbot", chatbot_reply)

        print(f"Chatbot: {chatbot_reply}")
        store_chat_data(db, user_info, query, chatbot_reply, emotion, confidence)

if __name__ == "__main__":
    main()


In [ ]:
import datetime
import firebase_admin
from firebase_admin import credentials, firestore
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Firebase Connection Setup
def setup_firebase():
    if not firebase_admin._apps:
        cred = credentials.Certificate("/content/project141-22a01-firebase-adminsdk-sgnh9-181e811b93.json")
        firebase_admin.initialize_app(cred)
    db = firestore.client()
    return db

# Emotion Detection Function
def detect_emotion(user_input):
    model_name = "j-hartmann/emotion-english-distilroberta-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
    emotion = emotion_labels[predicted_class]
    confidence = torch.softmax(logits, dim=1)[0][predicted_class].item()
    return emotion, confidence

# Helper function to send email
email_sent_users = set()

def send_email(user_email, city):
    global email_sent_users  # Reference the global set

    if user_email in email_sent_users:
        print(f"Email already sent to {user_email}.")
        return  # Exit the function
    helpline_numbers = {
        'mumbai': 'Mpower 1on1: 1800-120-820050',
        'pune': 'Connecting NGO: +91-9922001122',
        'nagpur': 'Jeevan Jyoti Suicide Prevention Centre: +91-712-2775555',
        'nashik': 'Connecting NGO: +91-9922001122',
        'aurangabad': 'Connecting NGO: +91-9922001122',
        'solapur': 'Connecting NGO: +91-9922001122',
        'kolhapur': 'Tele-MANAS: 14416',
        'thane': 'Tele-MANAS: 14416',
        'amravati': 'Tele-MANAS: 14416',
        'nanded': 'Tele-MANAS: 14416',
        'sangli': 'Tele-MANAS: 14416',
        'jalgaon': 'Tele-MANAS: 14416',
        'akola': 'Tele-MANAS: 14416',
        'latur': 'Tele-MANAS: 14416',
        'dhule': 'Tele-MANAS: 14416',
        'malegaon': 'Tele-MANAS: 14416',
        'bhusawal': 'Tele-MANAS: 14416',
        'beed': 'Tele-MANAS: 14416',
        'chandrapur': 'Tele-MANAS: 14416',
        'parbhani': 'Tele-MANAS: 14416',
        'yavatmal': 'Tele-MANAS: 14416',
        'osmanabad': 'Tele-MANAS: 14416',
        'navi mumbai': 'Mpower 1on1: 1800-120-820050',
        'kalyan': 'Mpower 1on1: 1800-120-820050',
        'ulhasnagar': 'Mpower 1on1: 1800-120-820050',
        'ambarnath': 'Mpower 1on1: 1800-120-820050',
        'dombivli': 'Mpower 1on1: 1800-120-820050',
        'bhiwandi': 'Mpower 1on1: 1800-120-820050',
        'ratnagiri': 'Mpower 1on1: 1800-120-820050',
        'satara': 'Mpower 1on1: 1800-120-820050',
        'sindhudurg': 'Mpower 1on1: 1800-120-820050',
        'wardha': 'Mpower 1on1: 1800-120-820050',
        'gondia': 'Mpower 1on1: 1800-120-820050',
        'bhandara': 'Mpower 1on1: 1800-120-820050',
        'washim': 'Mpower 1on1: 1800-120-820050',
        'hingoli': 'Mpower 1on1: 1800-120-820050',
        'gadchiroli': 'Mpower 1on1: 1800-120-820050',
    }

    if city not in helpline_numbers:
        helpline_numbers[city] = 'No helpline data available for your city.'

    sender_email = "pruthviraj.22420262@viit.ac.in"
    sender_password = "22420262"
    receiver_email = user_email

    subject = "Mental Health Support: Helpline Number"
    body = f"""
    Dear {user_email},

    We noticed that you're going through a tough time. Your emotional well-being is important to us.

    Here is the helpline number for your city ({city}):
    {helpline_numbers[city]}

    Please reach out to the helpline for any support you may need.

    Take care,
    Your Mental Health Chatbot
    """

    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        text = message.as_string()
        server.sendmail(sender_email, receiver_email, text)
        email_sent_users.add(receiver_email)
        print(f"Email sent successfully to {receiver_email}")
        server.quit()
    except Exception as e:
        print(f"Error sending email: {e}")

# Vector Database Creation
def create_vector_db():
    loader = DirectoryLoader("/content/data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

# QA Chain Setup
class ConversationHistory:
    def __init__(self):
        self.history = []

    def add_message(self, message_type, message):
        self.history.append((message_type, message))

    def get_context(self):
        context = "\n".join([f"{t}: {m}" for t, m in self.history[-5:]])  # Only keep last 5 messages for context
        return context if context else "No prior conversation history."

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    prompt_template = """You are Dr. James, a highly experienced and empathetic mental health professional having a one-on-one therapy session.
    Maintain a warm, professional tone while showing genuine concern for the patient's well-being.

    Important guidelines:
    - Address the patient's emotions with validation and understanding
    - Use therapeutic techniques like reflective listening and open-ended questions
    - Provide gentle guidance and coping strategies when appropriate
    - Maintain professional boundaries while being supportive
    - If you sense any risk of self-harm, emphasize the importance of seeking immediate help

    Previous conversation context:
    {context}

    Patient: {question}

    Dr. James:"""

    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

def store_user_data(db, user_info):
    user_ref = db.collection("users").document(user_info['name'])
    user_ref.set(user_info)

def store_chat_data(db, user_info, query, response, emotion, confidence):
    user_chat_ref = db.collection("users").document(user_info['name']).collection("chat_history")
    chat_entry = {
        "user_name": user_info['name'],
        "user_query": query,
        "chatbot_response": response,
        "emotion_detected": emotion,
        "confidence": confidence,
        "timestamp": datetime.datetime.now()
    }
    user_chat_ref.add(chat_entry)

def main():
    print("\n🌟 Welcome to Your Personal Therapy Session with Dr. James 🌟")
    print("\nInitializing your secure therapy environment...")
    db = setup_firebase()

    llm = ChatGroq(
        temperature=0.7,  # Increased for more natural, empathetic responses
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )

    db_path = "/content/chroma_db"
    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    print("\n📋 Before we begin, I'll need some information to provide you with the best support.")
    user_info = {}
    user_info['name'] = input("What name would you like me to call you? ")
    user_info['age'] = input("What is your age? ")
    user_info['city'] = input("Which city are you from? ")
    user_info['gender'] = input("What is your gender? ")
    user_info['email'] = input("Please share your email address for support resources: ")

    store_user_data(db, user_info)

    print(f"\nThank you, {user_info['name']}. I'm Dr. James, and I'm here to support you.")
    print("This is a safe space where you can share anything that's on your mind.")
    print("Remember, you can type 'exit' whenever you'd like to end our session.")

    conversation_history = ConversationHistory()
    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        query = input("\nHuman: ")
        if query.lower() == "exit":
            print("Chatbot: Take care of yourself, Goodbye!")
            break

        emotion, confidence = detect_emotion(query)
        print(f"Emotion Detected: {emotion} (Confidence: {confidence:.2f})")

        if emotion == "sadness" and confidence > 0.75:
            send_email(user_info['email'], user_info['city'])

        conversation_history.add_message("Human", query)
        context = conversation_history.get_context()
        response = qa_chain({"query": f"{context}\nUser: {query}"})
        chatbot_reply = response['result']
        conversation_history.add_message("Chatbot", chatbot_reply)

        print(f"Chatbot: {chatbot_reply}")
        store_chat_data(db, user_info, query, chatbot_reply, emotion, confidence)

if __name__ == "__main__":
    main()
